# Notebook 03: Embeddings e Ingestão no Pinecone

Este notebook realiza a ingestão dos dados processados no Pinecone. Aqui vamos:

1. **Conectar** com o índice Pinecone (biobyia)
2. **Configurar** o gerenciador de embeddings (Gemini ou Ollama)
3. **Gerar** embeddings para todos os chunks
4. **Ingerir** dados no Pinecone em lotes
5. **Verificar** ingestão bem-sucedida

## 📋 Pré-requisitos

- Notebook 02 executado com sucesso (chunks processados)
- Variáveis de ambiente configuradas (PINECONE_API_KEY, GEMINI_API_KEY, etc.)
- Índice Pinecone criado (biobyia)

## ⚠️ IMPORTANTE

- Este processo pode levar vários minutos dependendo do tamanho do dataset
- Certifique-se de ter créditos suficientes no Pinecone
- A ingestão é feita em lotes para otimizar performance

## 🔄 Ordem de Execução

Execute as células **sequencialmente** (de cima para baixo).


In [ ]:
# ============================================================================
# ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÕES
# ============================================================================

import sys
from pathlib import Path

# Adiciona o diretório raiz ao path
root_dir = Path.cwd().parent
sys.path.insert(0, str(root_dir))

# Importa módulos do pipeline RAG
from scripts.embeddings_manager import EmbeddingsManager
from scripts.pinecone_ingester import PineconeIngester
from config.settings import get_settings

# Carrega configurações
settings = get_settings()

# Valida configurações
is_valid, errors = settings.validate()
if not is_valid:
    print("❌ ERROS DE CONFIGURAÇÃO:")
    for error in errors:
        print(f"   - {error}")
    raise ValueError("Configurações inválidas")

print("✅ Bibliotecas importadas com sucesso!")
settings.print_config()


In [ ]:
# ============================================================================
# ETAPA 1: VERIFICAÇÃO DE CHUNKS (SE NÃO JÁ CARREGADOS)
# ============================================================================
# Se você executou o notebook 02, os chunks já estão na variável all_chunks
# Caso contrário, você precisará carregá-los

# Verifica se all_chunks existe
try:
    print(f"✅ Chunks já carregados: {len(all_chunks)} chunks prontos para ingestão")
except NameError:
    print("⚠️  Variável 'all_chunks' não encontrada!")
    print("   Execute o notebook 02 primeiro ou carregue os chunks processados")
    raise


In [ ]:
# ============================================================================
# ETAPA 2: INICIALIZAÇÃO DO GERENCIADOR DE EMBEDDINGS
# ============================================================================
# Configura o provider de embeddings (Gemini ou Ollama)
# Gemini é recomendado por ser mais rápido e confiável

print("=" * 80)
print("🔧 CONFIGURANDO EMBEDDINGS")
print("=" * 80)

# Cria gerenciador de embeddings
# Se não especificar provider, detecta automaticamente das configurações
embeddings_manager = EmbeddingsManager()

# Exibe informações sobre o provider
embedding_dim = embeddings_manager.get_embedding_dimension()
print(f"\nDimensão dos embeddings: {embedding_dim}")
print(f"Provider: {embeddings_manager.provider}")
print(f"Model: {embeddings_manager.model_name}")

print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 3: INICIALIZAÇÃO DO INGESTER DO PINECONE
# ============================================================================
# Conecta com o índice Pinecone e prepara para ingestão

print("=" * 80)
print("🔌 CONECTANDO COM PINECONE")
print("=" * 80)

# Cria ingester do Pinecone
ingester = PineconeIngester(
    embeddings_manager=embeddings_manager,
    index_name=settings.PINECONE_INDEX_NAME,
    namespace=settings.PINECONE_NAMESPACE,
    api_key=settings.PINECONE_API_KEY
)

print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 4: INGESTÃO NO PINECONE
# ============================================================================
# Esta é a etapa principal: gera embeddings e ingere todos os chunks
# no Pinecone em lotes otimizados
#
# ⚠️ ATENÇÃO: Este processo pode levar vários minutos!
# - Para ~10.000 chunks: ~10-15 minutos
# - Para ~100.000 chunks: ~2-3 horas
#
# O processo é feito em lotes para:
# - Otimizar uso de API
# - Evitar rate limiting
# - Permitir retry em caso de erro

print("=" * 80)
print("🚀 INICIANDO INGESTÃO NO PINECONE")
print("=" * 80)
print(f"Total de chunks a ingerir: {len(all_chunks)}")
print(f"Tamanho do lote: {settings.BATCH_SIZE}")
print(f"Total de lotes estimados: {(len(all_chunks) + settings.BATCH_SIZE - 1) // settings.BATCH_SIZE}")
print("\n⏳ Este processo pode levar vários minutos...")
print("=" * 80)

# Realiza a ingestão
ingestion_stats = ingester.ingest_chunks(
    all_chunks,
    batch_size=settings.BATCH_SIZE,
    show_progress=True
)

print("\n" + "=" * 80)
print("📊 ESTATÍSTICAS DA INGESTÃO")
print("=" * 80)
print(f"Total de chunks processados: {ingestion_stats['total_chunks']}")
print(f"Total de vetores inseridos: {ingestion_stats['total_vectors']}")
print(f"Total de lotes: {ingestion_stats['batches']}")
if ingestion_stats['errors']:
    print(f"Erros encontrados: {len(ingestion_stats['errors'])}")
    for error in ingestion_stats['errors'][:5]:  # Mostra apenas os 5 primeiros
        print(f"   - {error}")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 5: VERIFICAÇÃO DA INGESTÃO
# ============================================================================
# Verifica se os dados foram ingeridos corretamente fazendo uma query de teste

print("=" * 80)
print("🔍 VERIFICANDO INGESTÃO")
print("=" * 80)

# Importa função de query
from scripts.rag_query import query_medical_rag

# Faz uma query de teste
test_query = "mitochondria apoptosis"
print(f"Query de teste: '{test_query}'")
print("-" * 80)

try:
    results = query_medical_rag(
        test_query,
        embeddings_manager=embeddings_manager,
        top_k=3
    )
    
    print(f"✅ Query executada com sucesso!")
    print(f"   Resultados encontrados: {len(results)}")
    
    if results:
        print("\n📄 Primeiro resultado:")
        print(f"   Article ID: {results[0]['article_id']}")
        print(f"   Score: {results[0]['score']:.3f}")
        print(f"   Texto (primeiros 200 caracteres):")
        print(f"   {results[0]['text'][:200]}...")
    else:
        print("⚠️  Nenhum resultado encontrado. Verifique se a ingestão foi bem-sucedida.")
        
except Exception as e:
    print(f"❌ Erro ao executar query de teste: {e}")

print("=" * 80)


## ✅ Conclusão da Etapa 3

Neste notebook você:
- ✅ Configurou o gerenciador de embeddings
- ✅ Conectou com o índice Pinecone
- ✅ Gerou embeddings para todos os chunks
- ✅ Ingeriu dados no Pinecone em lotes
- ✅ Verificou a ingestão bem-sucedida

## 📌 Próximos Passos

Agora você está pronto para o próximo notebook:
- **Notebook 04**: Testes de queries RAG
- Validação de recuperação de contexto
- Exemplos práticos de uso
